All right reserved by #https://github.com/apache/incubator-mxnet/blob/master/docs/tutorials/onnx/fine_tuning_gluon.md

In [2]:
import json
import logging
import multiprocessing
import os
import tarfile

logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import gluon, nd, autograd
from mxnet.gluon.data.vision.datasets import ImageFolderDataset
from mxnet.gluon.data import DataLoader
import mxnet.contrib.onnx as onnx_mxnet
import numpy as np

%matplotlib inline

In [ ]:
image_folder = "images"
utils_file = "utils.py" # contain utils function to plot nice visualization
base_url = "https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/doc/tutorials/onnx/{}?raw=true"
mx.test_utils.download(base_url.format(utils_file), fname=utils_file)

from utils import *

In [ ]:
base_url = "https://s3.amazonaws.com/download.onnx/models/opset_3/"
current_model = "inception_v2"
model_folder = "model"
archive_file = "{}.tar.gz".format(current_model)
archive_path = os.path.join(model_folder, archive_file)
url = "{}{}".format(base_url, archive_file)
onnx_path = os.path.join(model_folder, current_model, 'model.onnx')

# Download the zipped model
mx.test_utils.download(url, dirname = model_folder)

# Extract the model
if not os.path.isdir(os.path.join(model_folder, current_model)):
    print('Extracting {} in {}...'.format(archive_path, model_folder))
    tar = tarfile.open(archive_path, "r:gz")
    tar.extractall(model_folder)
    tar.close()
    print('Model extracted.')


In [ ]:
#create data/train, data/val, data/test directories and paste images accordingly 
training_path = "data/train"
print(training_path)
testing_path = "data/val"
print(testing_path)

In [ ]:
EDGE = 224
SIZE = (EDGE, EDGE)
BATCH_SIZE = 16
NUM_WORKERS = 6

In [ ]:
def transform(image, label):
    resized = mx.image.resize_short(image, EDGE)
    cropped, crop_info = mx.image.center_crop(resized, SIZE)
    transposed = nd.transpose(cropped, (2,0,1))
    return transposed, label

In [ ]:
dataset_train = ImageFolderDataset(root=training_path)
dataset_test = ImageFolderDataset(root=testing_path)

In [ ]:
dataloader_train = DataLoader(dataset_train.transform(transform, lazy=False), batch_size=BATCH_SIZE, last_batch='rollover',
                              shuffle=True, num_workers=NUM_WORKERS)
dataloader_test = DataLoader(dataset_test.transform(transform, lazy=False), batch_size=BATCH_SIZE, last_batch='rollover',
                             shuffle=False, num_workers=NUM_WORKERS)
print("Train dataset: {} images, Test dataset: {} images".format(len(dataset_train), len(dataset_test)))

In [ ]:
categories = dataset_train.synsets
NUM_CLASSES = len(categories)
BATCH_SIZE = 32

In [ ]:
N = 100
plt.imshow((transform(dataset_train[N][0], 0)[0].asnumpy().transpose((1,2,0))))
plt.axis('off')
print(categories[dataset_train[N][1]])

In [ ]:
sym, arg_params, aux_params = onnx_mxnet.import_model(onnx_path)
def get_layer_output(symbol, arg_params, aux_params, layer_name):
    all_layers = symbol.get_internals()
    net = all_layers[layer_name+'_output']
    net = mx.symbol.Flatten(data=net)
    new_args = dict({k:arg_params[k] for k in arg_params if k in net.list_arguments()})
    new_aux = dict({k:aux_params[k] for k in aux_params if k in net.list_arguments()})
    return (net, new_args, new_aux)

sym.get_internals()


In [ ]:
new_sym, new_arg_params, new_aux_params = get_layer_output(sym, arg_params, aux_params, 'flatten0')

In [ ]:
ctx = mx.gpu() if mx.context.num_gpus() > 0 else mx.cpu()

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    pre_trained = gluon.nn.SymbolBlock(outputs=new_sym, inputs=mx.sym.var('data_0'))
net_params = pre_trained.collect_params()
for param in new_arg_params:
    if param in net_params:
        net_params[param]._load_init(new_arg_params[param], ctx=ctx)
for param in new_aux_params:
    if param in net_params:
        net_params[param]._load_init(new_aux_params[param], ctx=ctx)

In [ ]:
dense_layer = gluon.nn.Dense(NUM_CLASSES)
dense_layer.initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

In [ ]:
net = gluon.nn.HybridSequential()
net.hybridize()
with net.name_scope():
    net.add(pre_trained)
    net.add(dense_layer)

In [ ]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
LEARNING_RATE = 0.0001
WDECAY = 0.00001
MOMENTUM = 0.9

In [ ]:
# ctx = mx.gpu() 
net.collect_params().initialize(ctx=ctx)
trainer = gluon.Trainer(net.collect_params(), 'sgd', 
                        {'learning_rate': LEARNING_RATE,
                         'wd':WDECAY,
                         'momentum':MOMENTUM})

In [ ]:
def evaluate_accuracy_gluon(data_iterator, net):
    num_instance = 0
    sum_metric = nd.zeros(1,ctx=ctx, dtype=np.int32)
    for i, (data, label) in enumerate(data_iterator):
        data = data.astype(np.float32).as_in_context(ctx)
        label = label.astype(np.int32).as_in_context(ctx)
        output = net(data)
        prediction = nd.argmax(output, axis=1).astype(np.int32)
        num_instance += len(prediction)
        sum_metric += (prediction==label).sum()
    accuracy = (sum_metric.astype(np.float32)/num_instance)
    return accuracy.asscalar()


In [ ]:
%%time
print("Untrained network Test Accuracy: {0:.4f}".format(evaluate_accuracy_gluon(dataloader_test, net)))

In [ ]:
val_accuracy = 0
for epoch in range(10):
    for i, (data, label) in enumerate(dataloader_train):
        data = data.astype(np.float32).as_in_context(ctx)
        label = label.as_in_context(ctx)

        if i%20==0 and i >0:
            print('Batch [{0}] loss: {1:.4f}'.format(i, loss.mean().asscalar()))

        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

    nd.waitall() # wait at the end of the epoch    
    new_val_accuracy = evaluate_accuracy_gluon(dataloader_test, net)    
    print("Epoch [{0}] Test Accuracy {1:.4f} ".format(epoch, new_val_accuracy))

#     # We perform early-stopping regularization, to prevent the model from overfitting
#     if val_accuracy > new_val_accuracy:
#         print('Validation accuracy is decreasing, stopping training')
#         break
    val_accuracy = new_val_accuracy

In [ ]:
#Save net parameters
net.export(current_model, 100)
saved_model = current_model + '.params'
print (saved_model)
net.save_parameters(saved_model)

In [ ]:
def transform(img):
    return nd.array(np.expand_dims(np.transpose(img, (2,0,1)),axis=0).astype(np.float32), ctx=ctx)

In [ ]:
import os
import subprocess
import time
import cv2
import psutil

#This is the path of the images for testing
path = os.getcwd() + '/data/test'

#folders = []

files = os.listdir(path)
#print (files)

images = {}
false_assum = {}

for file in files:
   if 'hazard' in file:
      images.update({file : 'hazard'})
   elif 'clean' in file:
      images.update({file : 'clean'})

#print (images)

total_time = 0
for key in images:
   print ('\n\nImage name : ',key, 'Tag: ' , images[key], 'floor and the classification result is : ' )
   
   img = cv2.imread(path+'/'+key)
   img = cv2.resize(img,(224,224))
   #img = np.reshape(img,[1,224,224,3])
   #image = tf.cast(img, tf.float32)
   image = transform(img)
   start_time = time.time()
   classes = net(image)
   total_time = total_time + (time.time()-start_time)
   print(classes)
   pid = os.getpid()
   py = psutil.Process(pid)
   memoryUse = py.memory_info()[0]/2.**30  # memory use in GB...I think
   cpuusage = py.cpu_percent()
   print('memory use:', memoryUse)
   print('cpu use:', cpuusage)
   if classes[0][0] > classes[0][1]:
        res = 'clean'
        print('clean')
   else:
        res = 'hazard'
        print('hazard')
   
#    print("---%s seconds ---" % (time.time() - start_time))
   print("Image Name", key ,"classify as : ", res )
   if images[key] in res:
      images[key] = 1
   else:
      images[key] = 0
      false_assum.update({key: 0})
print("Total avg inference time: ", total_time/138)

print ("List of wrong assumtion : ", false_assum)
print ('########## Final result #########')
print ('Total right assumtion : ', sum(images.values()), '\nTotal worng assumtion : ', len(images)-sum(images.values()), '\nModel accuracy = ', round((sum(images.values())*100)/len(images), 3), '%')

In [ ]:
# Number of predictions to show (optional)
TOP_P = 1
images = os.listdir("images")

# Convert img to format expected by the network
def transform(img):
    return nd.array(np.expand_dims(np.transpose(img, (2,0,1)),axis=0).astype(np.float32), ctx=ctx)

# Load and transform the test images
caltech101_images_test = [plt.imread(os.path.join(image_folder, "{}".format(img))) for img in images]
caltech101_images_transformed = [transform(img) for img in caltech101_images_test]


In [ ]:
def run_batch(net, data):
    results = []
    for batch in data:
        outputs = net(batch)
        results.extend([o for o in outputs.asnumpy()])
    return np.array(results)

In [ ]:
#image should be 224*224
result = run_batch(net, caltech101_images_transformed)

In [ ]:
plot_predictions(caltech101_images_test, result, categories, TOP_P)